## 📦 Step 1: Install Dependencies

This cell installs all required packages. On Google Colab, this will take ~1-2 minutes.

In [1]:
# Install required packages
import sys
import warnings

# Suppress warnings for Python 3.13 namespace package compatibility
warnings.filterwarnings('ignore')

# Check if running on Colab
try:
    import google.colab
    IN_COLAB = True
    print("✅ Running on Google Colab")
except:
    IN_COLAB = False
    print(f"✅ Running locally (Python {sys.version_info.major}.{sys.version_info.minor})")

# Install packages
print("\n📦 Installing dependencies...")
print("   This may take 30-60 seconds...\n")

# Install with all dependencies explicitly
!pip install --upgrade google-generativeai protobuf ipywidgets

print("\n✅ Installation complete!")
print("\n⚠️  IMPORTANT: After running this cell for the first time:")
print("   1. Click 'Restart' in the kernel menu (or press Ctrl+Shift+P → 'Restart Kernel')")
print("   2. Then run cell 2 (skip cell 1 after restart)")
print("\n💡 If you see .pth warnings, they're harmless and can be ignored.")

✅ Running locally (Python 3.13)

📦 Installing dependencies...
   This may take 30-60 seconds...

  Using cached protobuf-6.33.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)

✅ Installation complete!

⚠️  IMPORTANT: After running this cell for the first time:
   1. Click 'Restart' in the kernel menu (or press Ctrl+Shift+P → 'Restart Kernel')
   2. Then run cell 2 (skip cell 1 after restart)

💡 If you see .pth warnings, they're harmless and can be ignored.


## 🔑 Step 2: Configure API Key

Enter your Google Gemini API key below. Get one free at: https://makersuite.google.com/app/apikey

In [2]:
import os
import getpass

# Check environment (in case kernel was restarted)
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

# Configure API Key
if IN_COLAB:
    # For Colab, use userdata secrets or input
    try:
        from google.colab import userdata
        GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
        print("✅ API key loaded from Colab secrets")
    except:
        # Fallback to manual input
        GEMINI_API_KEY = getpass.getpass("Enter your Gemini API Key: ")
else:
    # For local development, use environment variable or input
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
    if not GEMINI_API_KEY:
        GEMINI_API_KEY = getpass.getpass("Enter your Gemini API Key: ")

# Validate API key
if not GEMINI_API_KEY or len(GEMINI_API_KEY) < 20:
    raise ValueError("❌ Invalid API key. Please provide a valid Gemini API key.")

print("✅ API key configured successfully!")
print(f"🔐 Key preview: {GEMINI_API_KEY[:10]}...{GEMINI_API_KEY[-4:]}")

✅ API key configured successfully!
🔐 Key preview: AIzaSyBRzj...ICfk


## 🤖 Step 3: Initialize the Gemini Agent

This loads the AI agent that will converse with you to understand your requirements.

In [3]:
import google.generativeai as genai
from typing import Dict, List, Optional
import json
import re
from datetime import datetime

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)

print("🤖 Initializing Gemini Agent...")

# Test API connection
try:
    model = genai.GenerativeModel('gemini-2.5-flash')
    test_response = model.generate_content("Say 'Hello!'")
    print("✅ API connection successful!")
    print(f"📡 Model response: {test_response.text}")
except Exception as e:
    print(f"❌ API connection failed: {e}")
    raise

print("\n✅ Agent ready to start conversation!")

🤖 Initializing Gemini Agent...
✅ API connection successful!
📡 Model response: Hello!

✅ Agent ready to start conversation!


## 💬 Step 4: Define the Conversational Agent

This cell contains the core logic for the intelligent conversation system.

In [5]:
class BungalowDesignAgent:
    """Intelligent agent for gathering bungalow design requirements"""
    
    def __init__(self, api_key: str):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-2.5-flash')
        self.chat = None
        self.conversation_history = []
        self.extracted_requirements = {}
        self.questions_asked = set()
        
    def start_conversation(self) -> str:
        """Start a new conversation session"""
        system_prompt = """You are a professional architectural consultant helping customers design their dream bungalow. 
Your goal is to extract detailed requirements through natural, friendly conversation.

REQUIRED INFORMATION TO GATHER:
1. Bungalow type (single-story, duplex, villa, cottage)
2. Land area (in sq ft or sq meters)
3. Number of bedrooms (and preferred sizes like small/medium/large)
4. Number of bathrooms (specify if attached/common)
5. Living room requirement (yes/no, size preference)
6. Kitchen type (open/closed/semi-open)
7. Dining room (separate/combined with living room)
8. Parking (how many 2-wheelers and 4-wheelers)
9. Additional rooms: study room, guest room, prayer room, store room, laundry room
10. Balconies or terraces
11. Garden or outdoor space
12. Special preferences: 
    - Open floor plan vs traditional
    - Natural light preferences
    - Entrance direction (north/south/east/west)
    - Room adjacencies (e.g., master bedroom near bathroom)
    - Any accessibility features needed

CONVERSATION GUIDELINES:
- Start with a warm, professional greeting
- Ask 2-3 related questions at a time (don't overwhelm)
- If user is unsure, provide examples or typical options
- Be conversational and encouraging
- Acknowledge answers before moving to next topic
- After gathering all info, summarize everything for confirmation
- Keep responses concise but friendly

Start the conversation now with a welcoming message and begin asking about the basic requirements."""

        self.chat = self.model.start_chat(history=[])
        response = self.chat.send_message(system_prompt)
        
        agent_message = response.text
        self.conversation_history.append({"role": "agent", "content": agent_message})
        return agent_message
    
    def send_message(self, user_input: str) -> str:
        """Send user message and get response"""
        self.conversation_history.append({"role": "user", "content": user_input})
        
        # Add context reminder periodically
        context = ""
        if len(self.conversation_history) > 10 and len(self.conversation_history) % 5 == 0:
            context = "\n[Remember to keep questions focused and ask about any missing critical information.]"
        
        response = self.chat.send_message(user_input + context)
        agent_message = response.text
        
        self.conversation_history.append({"role": "agent", "content": agent_message})
        return agent_message
    
    def check_completion(self) -> tuple[bool, List[str]]:
        """Check if all required information is gathered"""
        conversation_text = "\n".join([f"{m['role']}: {m['content']}" for m in self.conversation_history])
        
        check_prompt = f"""Analyze this conversation and determine if ALL essential bungalow requirements have been gathered:

{conversation_text}

Required information:
1. Bungalow type
2. Land area
3. Number of bedrooms
4. Number of bathrooms
5. Living room
6. Kitchen type
7. Dining arrangement
8. Parking needs
9. Any additional rooms
10. Outdoor spaces

Respond in JSON format:
{{
    "complete": true/false,
    "missing_items": ["list of missing required information"],
    "confidence": "high/medium/low"
}}

Return ONLY the JSON."""

        try:
            response = self.model.generate_content(check_prompt)
            json_match = re.search(r'\{.*\}', response.text, re.DOTALL)
            if json_match:
                result = json.loads(json_match.group())
                return result.get("complete", False), result.get("missing_items", [])
        except:
            pass
        
        return False, []
    
    def extract_requirements(self) -> Dict:
        """Extract structured requirements from conversation"""
        conversation_text = "\n".join([f"{m['role']}: {m['content']}" for m in self.conversation_history])
        
        extraction_prompt = f"""Based on this conversation, extract ALL bungalow requirements into structured JSON.

{conversation_text}

Return this EXACT JSON structure (use null for missing info, but include all fields):
{{
    "bungalow_type": "single-story/duplex/villa/cottage or null",
    "land_area": {{"value": number or null, "unit": "sqft/sqm" or null}},
    "bedrooms": {{
        "count": number or null,
        "sizes": ["list of size preferences"] or [],
        "master_bedroom": true/false or null
    }},
    "bathrooms": {{
        "count": number or null,
        "attached": number or null,
        "common": number or null
    }},
    "living_room": {{
        "required": true/false,
        "size": "small/medium/large" or null,
        "combined_dining": true/false or null
    }},
    "kitchen": {{
        "type": "open/closed/semi-open" or null,
        "size": "small/medium/large" or null
    }},
    "dining_room": {{
        "type": "separate/combined/not-required" or null,
        "capacity": number or null
    }},
    "parking": {{
        "two_wheeler": number or 0,
        "four_wheeler": number or 0,
        "covered": true/false or null
    }},
    "additional_rooms": {{
        "study": true/false,
        "guest_room": true/false,
        "prayer_room": true/false,
        "store_room": true/false,
        "laundry": true/false,
        "other": [] or ["list of other rooms"]
    }},
    "outdoor_spaces": {{
        "balcony": true/false,
        "terrace": true/false,
        "garden": true/false,
        "patio": true/false,
        "lawn": true/false
    }},
    "preferences": {{
        "floor_plan_style": "open/traditional/mixed" or null,
        "natural_light": "high/medium/low" or null,
        "entrance_direction": "north/south/east/west/any" or null,
        "room_adjacencies": [] or ["list of preferences"],
        "special_requirements": [] or ["list of special needs"]
    }},
    "budget_range": "string or null",
    "timeline": "string or null",
    "additional_notes": "string or null"
}}

Return ONLY valid JSON, no markdown or extra text."""

        try:
            response = self.model.generate_content(extraction_prompt)
            json_text = response.text.strip()
            
            # Remove markdown code blocks if present
            json_text = re.sub(r'```json\s*', '', json_text)
            json_text = re.sub(r'```\s*$', '', json_text)
            
            # Extract JSON
            json_match = re.search(r'\{.*\}', json_text, re.DOTALL)
            if json_match:
                requirements = json.loads(json_match.group())
                self.extracted_requirements = requirements
                return requirements
            else:
                return {"error": "Could not extract JSON", "raw": response.text}
        except Exception as e:
            return {"error": f"Extraction failed: {str(e)}"}
    
    def get_summary(self) -> str:
        """Generate human-readable summary"""
        if not self.extracted_requirements:
            return "No requirements extracted yet."
        
        req = self.extracted_requirements
        lines = ["=" * 60, "🏠 BUNGALOW DESIGN REQUIREMENTS SUMMARY", "=" * 60, ""]
        
        # Basic info
        if req.get("bungalow_type"):
            lines.append(f"🏡 Type: {req['bungalow_type'].title()}")
        
        if req.get("land_area", {}).get("value"):
            land = req["land_area"]
            lines.append(f"📏 Land Area: {land['value']} {land.get('unit', 'sqft')}")
        
        lines.append("")
        lines.append("📐 ROOM CONFIGURATION:")
        
        # Rooms
        if req.get("bedrooms", {}).get("count"):
            br = req["bedrooms"]
            lines.append(f"  🛏️  Bedrooms: {br['count']}")
            if br.get("sizes"):
                lines.append(f"      Sizes: {', '.join(br['sizes'])}")
        
        if req.get("bathrooms", {}).get("count"):
            bt = req["bathrooms"]
            lines.append(f"  🚿 Bathrooms: {bt['count']}")
            if bt.get("attached"):
                lines.append(f"      Attached: {bt['attached']}")
        
        if req.get("living_room", {}).get("required"):
            lr = req["living_room"]
            lines.append(f"  🛋️  Living Room: {lr.get('size', 'Yes')}")
        
        if req.get("kitchen", {}).get("type"):
            lines.append(f"  🍳 Kitchen: {req['kitchen']['type'].title()}")
        
        if req.get("dining_room", {}).get("type"):
            lines.append(f"  🍽️  Dining: {req['dining_room']['type'].title()}")
        
        # Parking
        if req.get("parking"):
            parking = req["parking"]
            lines.append(f"  🚗 Parking: {parking.get('four_wheeler', 0)} cars, {parking.get('two_wheeler', 0)} bikes")
        
        # Additional rooms
        additional = req.get("additional_rooms", {})
        add_list = [k.replace('_', ' ').title() for k, v in additional.items() if v and k != 'other']
        if additional.get('other'):
            add_list.extend(additional['other'])
        if add_list:
            lines.append(f"  ➕ Additional: {', '.join(add_list)}")
        
        # Outdoor spaces
        outdoor = req.get("outdoor_spaces", {})
        outdoor_list = [k.title() for k, v in outdoor.items() if v]
        if outdoor_list:
            lines.append("")
            lines.append(f"🌳 OUTDOOR SPACES: {', '.join(outdoor_list)}")
        
        # Preferences
        pref = req.get("preferences", {})
        if any(pref.values()):
            lines.append("")
            lines.append("⚙️  PREFERENCES:")
            if pref.get("floor_plan_style"):
                lines.append(f"  • Style: {pref['floor_plan_style'].title()}")
            if pref.get("natural_light"):
                lines.append(f"  • Natural Light: {pref['natural_light'].title()}")
            if pref.get("entrance_direction"):
                lines.append(f"  • Entrance: {pref['entrance_direction'].title()}")
            if pref.get("room_adjacencies"):
                lines.append(f"  • Adjacencies: {', '.join(pref['room_adjacencies'][:3])}")
        
        if req.get("additional_notes"):
            lines.append("")
            lines.append(f"📝 NOTES: {req['additional_notes']}")
        
        lines.append("")
        lines.append("=" * 60)
        
        return "\n".join(lines)
    
    def save_session(self, filename: str = None):
        """Save conversation and requirements"""
        if not filename:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"bungalow_requirements_{timestamp}.json"
        
        data = {
            "timestamp": datetime.now().isoformat(),
            "conversation_history": self.conversation_history,
            "extracted_requirements": self.extracted_requirements
        }
        
        with open(filename, 'w') as f:
            json.dump(data, f, indent=2)
        
        return filename

# Initialize the agent
agent = BungalowDesignAgent(GEMINI_API_KEY)
print("✅ Agent class initialized!")

✅ Agent class initialized!


## 🎨 Step 5: Create Interactive Chat Interface

Now let's create a beautiful, interactive chatbot UI!

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time

# Create UI components
output_area = widgets.Output(layout=widgets.Layout(
    height='400px', 
    width='100%',
    border='2px solid #4CAF50',
    border_radius='10px',
    padding='10px',
    overflow_y='auto'
))

input_box = widgets.Text(
    placeholder='Type your message here...',
    layout=widgets.Layout(width='70%'),
    style={'description_width': 'initial'}
)

send_button = widgets.Button(
    description='Send',
    button_style='success',
    layout=widgets.Layout(width='15%'),
    icon='paper-plane'
)

new_chat_button = widgets.Button(
    description='New Chat',
    button_style='warning',
    layout=widgets.Layout(width='15%'),
    icon='refresh'
)

extract_button = widgets.Button(
    description='📋 Extract Requirements',
    button_style='info',
    layout=widgets.Layout(width='100%', margin='10px 0px'),
    icon='check'
)

# Style for chat messages
def format_message(role: str, content: str) -> str:
    """Format chat message with styling"""
    if role == "agent":
        color = "#E3F2FD"
        icon = "🤖"
        align = "left"
    else:
        color = "#F1F8E9"
        icon = "👤"
        align = "right"
    
    return f"""
    <div style="text-align: {align}; margin: 10px 0;">
        <div style="display: inline-block; background-color: {color}; 
                    padding: 10px 15px; border-radius: 15px; max-width: 80%;
                    box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
            <strong>{icon} {role.title()}:</strong><br>
            {content}
        </div>
    </div>
    """

# Chat state
chat_started = False
processing = False

def display_message(role: str, content: str):
    """Display a chat message"""
    with output_area:
        display(HTML(format_message(role, content)))
        output_area.layout.overflow_y = 'auto'

def start_chat(b=None):
    """Start new conversation"""
    global chat_started, agent, processing
    
    if processing:
        return
    
    processing = True
    output_area.clear_output()
    
    with output_area:
        display(HTML("<h3 style='color: #4CAF50; text-align: center;'>🏠 Starting Conversation...</h3>"))
    
    try:
        response = agent.start_conversation()
        output_area.clear_output()
        display_message("agent", response)
        chat_started = True
        input_box.disabled = False
        send_button.disabled = False
    except Exception as e:
        with output_area:
            display(HTML(f"<p style='color: red;'>Error: {str(e)}</p>"))
    
    processing = False

def send_message(b=None):
    """Send user message"""
    global processing
    
    if processing or not chat_started:
        return
    
    user_input = input_box.value.strip()
    if not user_input:
        return
    
    processing = True
    input_box.disabled = True
    send_button.disabled = True
    
    # Display user message
    display_message("user", user_input)
    input_box.value = ""
    
    # Get agent response
    try:
        with output_area:
            display(HTML("<p style='color: #888; font-style: italic;'>Agent is typing...</p>"))
        
        response = agent.send_message(user_input)
        
        # Remove typing indicator
        with output_area:
            # Clear only the typing message
            output_children = output_area.outputs
            if output_children:
                output_area.outputs = output_children[:-1]
        
        display_message("agent", response)
        
    except Exception as e:
        display_message("agent", f"Sorry, I encountered an error: {str(e)}")
    
    input_box.disabled = False
    send_button.disabled = False
    processing = False
    input_box.focus()

def extract_requirements_ui(b=None):
    """Extract and display requirements"""
    global processing
    
    if processing or not chat_started:
        with output_area:
            display(HTML("<p style='color: orange;'>Please start a conversation first!</p>"))
        return
    
    processing = True
    extract_button.disabled = True
    
    with output_area:
        display(HTML("<h3 style='color: #2196F3;'>📋 Extracting Requirements...</h3>"))
    
    try:
        requirements = agent.extract_requirements()
        
        if "error" in requirements:
            display(HTML(f"<p style='color: red;'>Error: {requirements['error']}</p>"))
        else:
            summary = agent.get_summary()
            display(HTML(f"<pre style='background: #f5f5f5; padding: 15px; border-radius: 5px;'>{summary}</pre>"))
            
            # Save to file
            filename = agent.save_session()
            display(HTML(f"<p style='color: green;'>✅ Requirements saved to: <strong>{filename}</strong></p>"))
            
    except Exception as e:
        with output_area:
            display(HTML(f"<p style='color: red;'>Error: {str(e)}</p>"))
    
    extract_button.disabled = False
    processing = False

# Attach event handlers
send_button.on_click(send_message)
new_chat_button.on_click(start_chat)
extract_button.on_click(extract_requirements_ui)

# Handle Enter key
def handle_submit(sender):
    send_message()

input_box.on_submit(handle_submit)

# Initially disable input until chat starts
input_box.disabled = True
send_button.disabled = True

print("✅ UI Components Created!")

✅ UI Components Created!


## 🚀 Step 6: Launch the Chatbot!

Click **"New Chat"** to start the conversation. The AI agent will guide you through the requirements.

**Tips:**
- Answer naturally - you don't need to be technical
- If unsure, the agent will provide suggestions
- You can ask the agent to clarify questions
- After the conversation, click **"Extract Requirements"** to see the structured output

In [7]:
# Display the chat interface
display(HTML("""
<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
            padding: 20px; border-radius: 10px; text-align: center; margin-bottom: 20px;'>
    <h1 style='color: white; margin: 0;'>🏠 Bungalow Design Assistant</h1>
    <p style='color: white; margin: 5px 0;'>Let's design your dream home together!</p>
</div>
"""))

# Button container
button_box = widgets.HBox([new_chat_button, widgets.Box(layout=widgets.Layout(width='70%'))])
display(button_box)

# Chat area
display(output_area)

# Input container
input_container = widgets.HBox([input_box, send_button])
display(input_container)

# Extract button
display(extract_button)

# Instructions
display(HTML("""
<div style='background: #E8F5E9; padding: 15px; border-radius: 5px; margin-top: 20px;'>
    <h4>📌 Instructions:</h4>
    <ol>
        <li>Click <strong>"New Chat"</strong> to start</li>
        <li>Answer the agent's questions naturally</li>
        <li>Press <strong>Enter</strong> or click <strong>"Send"</strong> to send messages</li>
        <li>After gathering all requirements, click <strong>"Extract Requirements"</strong></li>
        <li>Your requirements will be saved as a JSON file for Phase 2!</li>
    </ol>
</div>
"""))

print("\n✅ Chat interface ready! Click 'New Chat' to begin.")

Output(layout=Layout(border_bottom='2px solid #4CAF50', border_left='2px solid #4CAF50', border_right='2px sol…

Button(button_style='info', description='📋 Extract Requirements', icon='check', layout=Layout(margin='10px 0px…


✅ Chat interface ready! Click 'New Chat' to begin.


In [9]:
# Alternative: Text-based interface (if widgets don't work)
print("🏠 BUNGALOW DESIGN ASSISTANT - Text Mode")
print("=" * 60)
print("\nStarting conversation...")
print("\n" + "=" * 60)

# Start conversation
try:
    response = agent.start_conversation()
    print(f"\n🤖 Agent:\n{response}\n")
    print("=" * 60)
    
    # Interactive loop
    while True:
        user_input = input("\n👤 You: ").strip()
        
        if not user_input:
            continue
            
        if user_input.lower() in ['quit', 'exit', 'done', 'finish']:
            print("\n✅ Ending conversation...")
            break
        
        print("\n🤖 Agent is typing...\n")
        response = agent.send_message(user_input)
        print(f"🤖 Agent:\n{response}\n")
        print("=" * 60)
        
except KeyboardInterrupt:
    print("\n\n✅ Conversation ended.")
    
print("\n💡 Type 'extract' in the next cell to extract requirements!")

🏠 BUNGALOW DESIGN ASSISTANT - Text Mode

Starting conversation...



✅ Conversation ended.

💡 Type 'extract' in the next cell to extract requirements!


## ⚠️ Widgets Not Showing?

If you don't see the buttons above, try:
1. **Install Jupyter Widget Extension** in VS Code
2. **Use Google Colab** instead (upload this notebook there)
3. **Use the text-based interface below** (no widgets needed)

## 📊 Step 7: View Extracted Requirements (Optional)

After extracting requirements, you can view the structured JSON data here.

In [10]:
# View extracted requirements
if agent.extracted_requirements:
    print("📋 Extracted Requirements (JSON):\n")
    print(json.dumps(agent.extracted_requirements, indent=2))
else:
    print("No requirements extracted yet. Complete the conversation and click 'Extract Requirements' first.")

📋 Extracted Requirements (JSON):

{
  "bungalow_type": "single-story",
  "land_area": {
    "value": null,
    "unit": null
  },
  "bedrooms": {
    "count": 2,
    "sizes": [
      "one larger",
      "one smaller"
    ],
    "master_bedroom": true
  },
  "bathrooms": {
    "count": 2,
    "attached": 1,
    "common": 1
  },
  "living_room": {
    "required": true,
    "size": "large",
    "combined_dining": true
  },
  "kitchen": {
    "type": "open",
    "size": null
  },
  "dining_room": {
    "type": "combined",
    "capacity": null
  },
  "parking": {
    "two_wheeler": 2,
    "four_wheeler": 1,
    "covered": null
  },
  "additional_rooms": {
    "study": false,
    "guest_room": false,
    "prayer_room": false,
    "store_room": false,
    "laundry": false,
    "other": []
  },
  "outdoor_spaces": {
    "balcony": true,
    "terrace": false,
    "garden": false,
    "patio": false,
    "lawn": false
  },
  "preferences": {
    "floor_plan_style": "open",
    "natural_light": "h

## 💾 Step 8: Export Requirements for Phase 2

Run this cell to save your requirements in a format ready for floor plan generation.

In [11]:
import os
from datetime import datetime

def export_for_phase2():
    """Export requirements in Phase 2 compatible format"""
    if not agent.extracted_requirements:
        print("❌ No requirements to export. Please extract requirements first.")
        return
    
    # Create output directory
    os.makedirs("generated_requirements", exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save full requirements
    full_path = f"generated_requirements/requirements_{timestamp}.json"
    with open(full_path, 'w') as f:
        json.dump(agent.extracted_requirements, indent=2, fp=f)
    
    # Save conversation history
    history_path = f"generated_requirements/conversation_{timestamp}.json"
    with open(history_path, 'w') as f:
        json.dump({
            "timestamp": datetime.now().isoformat(),
            "conversation": agent.conversation_history,
            "requirements": agent.extracted_requirements
        }, indent=2, fp=f)
    
    # Display summary
    print("✅ Requirements exported successfully!\n")
    print(f"📁 Files created:")
    print(f"   1. {full_path}")
    print(f"   2. {history_path}")
    print(f"\n📊 Summary:")
    print(agent.get_summary())
    print(f"\n🚀 Ready for Phase 2: Floor Plan Generation!")
    
    return full_path, history_path

# Export button
export_btn = widgets.Button(
    description='💾 Export for Phase 2',
    button_style='success',
    layout=widgets.Layout(width='100%', height='50px'),
    icon='download'
)

def on_export_click(b):
    with widgets.Output():
        export_for_phase2()

export_btn.on_click(on_export_click)
display(export_btn)

print("\n💡 Click the button above to export requirements for Phase 2!")

Button(button_style='success', description='💾 Export for Phase 2', icon='download', layout=Layout(height='50px…


💡 Click the button above to export requirements for Phase 2!


---

## 🎉 Phase 1 Complete!

You've successfully gathered all requirements for your bungalow design. 

### Next Steps:
1. ✅ Requirements are saved in `generated_requirements/` folder
2. 🚀 Move to **Phase 2 Notebook** for floor plan generation using House-GAN
3. 🎨 Phase 3 will generate 3D visualizations

### Need to Make Changes?
- Click "New Chat" to start over
- Or manually edit the JSON file and proceed to Phase 2

---

**📞 Support:** If you encounter any issues, check that your Gemini API key is valid and you have internet connection.